Import the required libraries

In [1]:
from jupyter_core.paths import jupyter_path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import os
import glob
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import timedelta

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import lightgbm as lgb # Our ML library

In [3]:
#Cross validation libraries
from sklearn.model_selection import TimeSeriesSplit

In [4]:
import joblib

Read all CSVs with stocks data and append to one big file

In [5]:
#os.chdir("/Users/olegkazanskyi/Documents/GitHub/Trading/CSVs")
os.chdir("C:/Users/oleg.kazanskyi/OneDrive - Danaher/Documents/Trading/SP500_CSVs_01032023")
filepaths = [f for f in os.listdir("./") if f.endswith('.csv')]
df = pd.DataFrame()
for i in filepaths:
    iterate_df = pd.DataFrame()
    iterate_df = pd.read_csv(i, encoding= 'unicode_escape')
    iterate_df["stock"] = i[:-4]
    df = pd.concat([df,iterate_df])
#df = pd.concat(map(pd.read_csv, filepaths))

df = df[df.close.notna()]
os.chdir("C:/Users/oleg.kazanskyi/OneDrive - Danaher/Documents/Trading/ML_part/EOD")
df.to_csv("THE_FINAL_DATASET_2023.csv")

record the dataframe to speed up the future reading process

os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python/")
df.to_csv("THE_FINAL_DATASET_2023.csv")

In [6]:
#df['YoY_DY'] = 100*df['DY'].pct_change(periods = -365)

In [7]:
df.shape

(702978, 65)

In [8]:
df.duplicated().sum()

0

Set up Date column as an index columns

In [9]:
df["date"] = pd.to_datetime(df["date"])

Let's check how much empty values we have by column

In [10]:
zeroes = df.isnull().sum()
print(zeroes[zeroes>0])
del zeroes

days_after_earnings_report     1420
LTDE                           7610
DE                             1182
DPR                           66581
Acc_Rec_Pay_Ration             1999
DY                            66581
PEG_Forward                    2941
EPS_surprise                   1420
EPS_YoY_Growth                 1420
EPS_QoQ_frcst_diff             1420
EPS_1Y_exp_Change              4361
YoY_DPR                       61477
YoY_AR_Ration                  1483
YoY_DY                        61477
EPS_1Y_exp_Change_QoQ         52830
future_15dprice_change         5390
future_30dprice_change        10780
future_60dprice_change        21560
future_90dprice_change        32340
future_120dprice_change       43113
future_150dprice_change       53852
VIX_DoD                         490
VIX_WoW                        2450
VIX_MoM                       10780
dtype: int64


### Dealing with "EPS" column

There are many null values in the "current_ratio" column. Let's see the stocks where it happens and then decide what to do

In [11]:
df[df.EPS_surprise.isnull()].stock.unique()

array(['APA', 'DD', 'ETSY', 'FOX', 'HSIC', 'INVH', 'L', 'NI', 'PODD'],
      dtype=object)

After checking the original files, it's apparently the null values comes from the older data.

Before 2017 our API did not provide us info about earinings. 

We can drop the reocrds with 'nan' values

we will remove only 10k records 

In [12]:
df = df[df.EPS_surprise.notnull()]

Let's check the dataset again

In [13]:
def check_null_cols(df):
    zeroes = df.isnull().sum()
    print(zeroes[zeroes>0])
    
check_null_cols(df)   

LTDE                        7610
DE                          1182
DPR                        66492
Acc_Rec_Pay_Ration          1999
DY                         66492
PEG_Forward                 2941
EPS_1Y_exp_Change           2941
YoY_DPR                    61388
YoY_AR_Ration               1483
YoY_DY                     61388
EPS_1Y_exp_Change_QoQ      52830
future_15dprice_change      5357
future_30dprice_change     10714
future_60dprice_change     21445
future_90dprice_change     32181
future_120dprice_change    42910
future_150dprice_change    53605
VIX_DoD                      490
VIX_WoW                     2450
VIX_MoM                    10780
dtype: int64


### Dealing with the null values in the "dividends" column

About 30% of data is affected by null values in the dividends columns.
Let's check how many companies are in the list

In [14]:
print("Number of companies with zero values in dividends",len(df[df['DY'].isnull()].stock.unique()))

Number of companies with zero values in dividends 51


This companies do not pay dividends, we can replace payments to "0"

In [15]:
df['DY'].fillna(0, inplace=True)
df['YoY_DPR'].fillna(0, inplace=True)
df['DPR'].fillna(0, inplace=True)
df['YoY_DY'].fillna(0, inplace=True)                   

Let's mark the companies with no dividends with the categorical columnm

In [16]:
df.loc[df['DY'] == 0,"Pays_Divds"] = 1
df.loc[df['DY'] != 0,"Pays_Divds"] = 0

In [17]:
temporary = df[df['DY'] == 0].copy()

print("Doesn't pay dividends")
for column in temporary.columns:
    if temporary[column].nunique() < 20:
        print(column, temporary[column].nunique())
        
temporary = df[df['DY'] != 0].copy()

print("\nPay dividends")
for column in temporary.columns:
    if temporary[column].nunique() < 20:
        print(column, temporary[column].nunique())
    

Doesn't pay dividends
sector 7
industry 14
DPR 1
DY 1
Piotroski_Score 9
YoY_DPR 1
YoY_DY 1
Pays_Divds 1

Pay dividends
sector 11
Piotroski_Score 9
Pays_Divds 1


### Dealing with YoY

The YoY variables with null values are caused by YoY calculation of the rows without historical data.
Let's drop these

In [18]:
df = df[df.YoY_CR.notnull()]

Let's check the dataset again

In [19]:
check_null_cols(df)   

LTDE                        7610
DE                          1182
Acc_Rec_Pay_Ration          1999
PEG_Forward                 2941
EPS_1Y_exp_Change           2941
YoY_AR_Ration               1483
EPS_1Y_exp_Change_QoQ      52830
future_15dprice_change      5357
future_30dprice_change     10714
future_60dprice_change     21445
future_90dprice_change     32181
future_120dprice_change    42910
future_150dprice_change    53605
VIX_DoD                      490
VIX_WoW                     2450
VIX_MoM                    10780
dtype: int64


In [20]:
df.shape

(701558, 66)

### Dealing with debt ratio

Let's check the companies that contain null values

In [21]:
df[df['DE'].isnull()].stock.unique()

array(['FFIV', 'FTNT'], dtype=object)

It seems these companies had zero debt for some quarters.

Let's replace with "0"

In [22]:
df['DE'].fillna(0, inplace=True)
df['LTDE'].fillna(0, inplace=True)

In [23]:
check_null_cols(df)   

Acc_Rec_Pay_Ration          1999
PEG_Forward                 2941
EPS_1Y_exp_Change           2941
YoY_AR_Ration               1483
EPS_1Y_exp_Change_QoQ      52830
future_15dprice_change      5357
future_30dprice_change     10714
future_60dprice_change     21445
future_90dprice_change     32181
future_120dprice_change    42910
future_150dprice_change    53605
VIX_DoD                      490
VIX_WoW                     2450
VIX_MoM                    10780
dtype: int64


### Dealing with VIX

Let's remove null values for VIX as these are related to the historical calculations

In [24]:
df = df[df.VIX_MoM.notnull()]

In [25]:
check_null_cols(df) 

Acc_Rec_Pay_Ration          1933
PEG_Forward                 2637
EPS_1Y_exp_Change           2637
YoY_AR_Ration               1461
EPS_1Y_exp_Change_QoQ      42645
future_15dprice_change      5357
future_30dprice_change     10714
future_60dprice_change     21445
future_90dprice_change     32174
future_120dprice_change    42869
future_150dprice_change    53561
dtype: int64


### Dealing with Accounts Payable

In [26]:
df[df['Acc_Rec_Pay_Ration'].isnull()].stock.unique()

array(['AVB', 'CINF', 'MAA'], dtype=object)

It seems these companies had zero Accounts receivables for some quarters

In [27]:
df['Acc_Rec_Pay_Ration'].fillna(0, inplace=True)

### Dealing with PEG

In [28]:
df[df['PEG_Forward'].isnull()].stock.unique()

array(['CZR', 'FOXA', 'KHC', 'LHX', 'PLD', 'SEDG'], dtype=object)

We can drop the rows as the effect would be unsignificant

In [29]:
df = df[df.PEG_Forward.notnull()]

In [30]:
df[df['PEG_Backwards'].isnull()].stock.unique()

array([], dtype=object)

In [31]:
df = df[df.PEG_Backwards.notnull()]

In [32]:
df = df[df.EPS_1Y_exp_Change.notnull()]

In [33]:
check_null_cols(df) 

YoY_AR_Ration               1461
EPS_1Y_exp_Change_QoQ      40008
future_15dprice_change      5357
future_30dprice_change     10714
future_60dprice_change     21445
future_90dprice_change     32174
future_120dprice_change    42869
future_150dprice_change    53561
dtype: int64


### Dealing with Industry

In [34]:
df[df['sector'].isnull()].stock.unique()

array([], dtype=object)

Let's fill the sector and industry values for the GEN company

In [35]:
df.loc[df['stock'] == 'GEN','sector'] = 'Information Technology'
df.loc[df['stock'] == 'GEN','industry'] = 'Software & Services'

In [36]:
check_null_cols(df) 

YoY_AR_Ration               1461
EPS_1Y_exp_Change_QoQ      40008
future_15dprice_change      5357
future_30dprice_change     10714
future_60dprice_change     21445
future_90dprice_change     32174
future_120dprice_change    42869
future_150dprice_change    53561
dtype: int64


In [37]:
df = df[df.YoY_AR_Ration.notnull()]

### Working with data types

Now we can keep only the necessery columns

In [38]:
categoric_columns = df.select_dtypes(include='object').columns
for col in categoric_columns:
    if col == "stock":
        continue
    print(f"column {col}, data: \n {df[col].unique()}")

column sector, data: 
 ['Health Care' 'Industrials' 'Consumer Discretionary'
 'Information Technology' 'Financials' 'Consumer Staples' 'Utilities'
 'Materials' 'Real Estate' 'Energy' 'Communication Services']
column industry, data: 
 ['Pharmaceuticals, Biotechnology & Life Sciences' 'Transportation'
 'Retailing' 'Technology Hardware & Equipment'
 'Health Care Equipment & Services' 'Insurance' 'Software & Services'
 'Semiconductors & Semiconductor Equipment' 'Food, Beverage & Tobacco'
 'Utilities' 'Materials' 'Capital Goods' 'Diversified Financials'
 'Real Estate' 'Energy' 'Automobiles & Components' 'Media & Entertainment'
 'Banks' 'Consumer Services' 'Household & Personal Products'
 'Food & Staples Retailing' 'Commercial & Professional Services'
 'Consumer Durables & Apparel' 'Telecommunication Services']
column Piotroski_Score, data: 
 [5 7 6 8 4 9 2 3 1]


In [39]:
#Number of stocks per industry
df[['sector','industry','stock']].groupby('industry').stock.nunique()

industry
Automobiles & Components                           5
Banks                                             15
Capital Goods                                     48
Commercial & Professional Services                 9
Consumer Durables & Apparel                       13
Consumer Services                                 15
Diversified Financials                            26
Energy                                            23
Food & Staples Retailing                           4
Food, Beverage & Tobacco                          23
Health Care Equipment & Services                  37
Household & Personal Products                      5
Insurance                                         21
Materials                                         28
Media & Entertainment                             20
Pharmaceuticals, Biotechnology & Life Sciences    27
Real Estate                                       28
Retailing                                         22
Semiconductors & Semiconductor Equipm

In [40]:
#Number of stocks per sector
df[['sector','industry','stock']].groupby('sector').stock.nunique()

sector
Communication Services    23
Consumer Discretionary    55
Consumer Staples          32
Energy                    23
Financials                62
Health Care               64
Industrials               71
Information Technology    74
Materials                 28
Real Estate               28
Utilities                 29
Name: stock, dtype: int64

We can drop "Stocks" and "Industry" columns as there are too many unique values that block us from generalizing the data. 

In [41]:
#df.drop(["industry", "stock"], axis = 1, inplace = True)
#df.drop(["industry"], axis = 1, inplace = True)

Let's also check if there are any infinite numbers that can cause same trublesas nan

In [42]:
df.replace([np.inf, -np.inf], np.nan, inplace = True)

check_null_cols(df) 

EPS_YoY_Growth                51
EPS_QoQ_frcst_diff           122
EPS_1Y_exp_Change            206
EPS_1Y_exp_Change_QoQ      40281
future_15dprice_change      5346
future_30dprice_change     10692
future_60dprice_change     21401
future_90dprice_change     32108
future_120dprice_change    42781
future_150dprice_change    53451
dtype: int64


In [43]:
df = df[df.EPS_1Y_exp_Change.notnull() & df.EPS_YoY_Growth.notnull() & df.EPS_QoQ_frcst_diff.notnull() & df.EPS_1Y_exp_Change_QoQ.notnull()]

In [44]:
check_null_cols(df)

future_15dprice_change      5346
future_30dprice_change     10692
future_60dprice_change     21391
future_90dprice_change     32076
future_120dprice_change    42746
future_150dprice_change    53416
dtype: int64


In [46]:
#df = df[df['Fed_Balance_YoY'].notnull()]

### Data Cleaning is Over!

## Trimming the data to avoid overfitting

 Our dataset have the range of dates that are very close to each other. 

The changes in trading value are rarely very different between today and yesterday. 

There are exemptions but mostly these are connected with big surprises, and in any case we would notice the change even if we track every other day or every 4th day.

It means that removing part of the dataset should help us in building more generalized model since we will be looking for trend but not for matching values.


Let's do this, let's remove 4/5 of the dataset.

That means we will keep only one day of data per week. It will help us to generalize the dataset

print("Old dataframe shape:", df.shape)
df_compact = df.iloc[::5, :]
print("New dataframe shape:", df_compact.shape)

The final dataset is 131K rows long with 53 variables and 6 targets(price after 15 days, 30 days, 60 days, 90 days, 120 days and 150 days)

We can remove the bigger dataset, but let's save it to the file before doing so.

In [47]:
os.chdir("C:/Users/oleg.kazanskyi/OneDrive - Danaher/Documents/Trading/ML_Part/EOD")
df.to_csv("full_cleaned_dataframe_2023.csv", index = False, header = True)
del df

Lets' save the shorter version of the dataframe as well so we can get it faster when required

os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python/ML_Part/EOD")
df_compact.to_csv("shorter_cleaned_dataframe_2023.csv", index = False, header = True)